In [ ]:
### Check Working directory

import os
print("Working dir: ", os.getcwd())

os.chdir(os.pardir)         # change working dir to parents dir
print("Changed dir: ", os.getcwd())

In [ ]:
# view sim result

import pandas as pd

file_path = 'result/res_cosine_club.xlsx'
df = pd.read_excel(file_path, sheet_name = 'Sheet1')

sorted_df = df.sort_values(by='Cosine Similarity', ascending=False)

print(sorted_df)

In [ ]:
# club divide result
df_group = pd.read_excel(file_path, sheet_name = 'Sheet4')

print(df_group)

In [21]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# read raw data
df_raw = pd.read_excel('raw_data/refined_data.xlsx')
print(df_raw)

file_path = 'result/res_cosine_club.xlsx'
df_club = pd.read_excel(file_path, sheet_name = 'Sheet4')

# Function to calculate cosine similarity
def cos_sim(data):

    data = data.iloc[:, 1:].values

    # Calculate cosine similarity
    cosine_similarity_matrix = cosine_similarity(data)

    similarity_result = []

    # for loop for cosine similarity
    for i in range(len(cosine_similarity_matrix)):
        for j in range(i + 1, len(cosine_similarity_matrix)):
            similarity_result.append([i + 1, j + 1, cosine_similarity_matrix[i, j]])

    # Return result as a DataFrame
    return pd.DataFrame(similarity_result, columns=["Row1", "Row2", "Cosine Similarity"])


# Call function and display result
indi_df = cos_sim(df_raw)
print("=== indivisual ===")
print(indi_df)

print("=== club div ===")
div_df = cos_sim(df_club)
print(div_df)


                                   ID   EXT   NEU   AGR   CON   OPN
0    00419a4c96b32cd63b2c7196da761274  1.90  4.15  3.15  4.65  3.45
1    02c37028a782cfda660c7243e45244bb  2.05  4.00  2.20  2.20  3.75
2    03133a828cd0cf52e3752813ce5d818f  3.84  3.55  2.85  4.32  3.45
3    03e6c4eca4269c183fa0e1780f73faba  3.20  3.60  3.85  4.35  4.80
4    06b055f8e2bca96496514891057913c3  2.85  2.35  3.35  4.70  3.35
..                                ...   ...   ...   ...   ...   ...
245  f83aa7290cf3ae8ed75d6aee5ebbb559  2.65  1.70  3.95  3.00  3.15
246  fbe5aa478508d1dc931427ade5d9e1b4  3.25  2.75  2.25  4.00  4.00
247  fc1c9fb6c64740edcbbf8cfe9dde8b02  3.55  2.25  3.90  2.80  4.00
248  fe22087986fdcc65939c793fe0ec90a9  3.45  2.55  3.15  3.50  2.90
249  fffafe151f07a30a0ede2038a897b680  3.05  1.80  3.35  2.95  4.35

[250 rows x 6 columns]
=== indivisual ===
       Row1  Row2  Cosine Similarity
0         1     2           0.951905
1         1     3           0.966918
2         1     4           0.

In [28]:
# indi_df and div_df

# raw_data
print(df_raw.iloc[0:1, 1:])



   EXT   NEU   AGR   CON   OPN
0  1.9  4.15  3.15  4.65  3.45
